In [ ]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("yarn")\
        .appName("movie")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.sql.warehouse.dir", "hdfs:/user/bigdata/repository/database/") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "4") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

In [ ]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
df.show(truncate=False)

In [ ]:
# df.filter("primaryProfession in ('soundtrack')").show()
df = (df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N'  ")
        .withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
        .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))

        

)
dfs = df.select(F.col("nconst"),
                F.col("primaryName"),
                F.col("birthYear"),
                F.col("deathYear"),
                F.explode(F.col("profession_array")).alias("profession"),
                F.explode(F.col("knownForTitles_array")).alias("Titles"))

                
dfs.show()

#123.627.002
#11.845.112
#357.801


In [ ]:
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

def dp_etl(df: DataFrame) -> DataFrame :

    dfs = (df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N' ")
                .withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
                .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))
                .select(F.col("nconst"),
                        F.col("primaryName"),
                        F.col("birthYear"),
                        F.col("deathYear"),
                        F.explode(F.col("profession_array")).alias("profession"),
                        F.explode(F.col("knownForTitles_array")).alias("Titles"))
    )
    dffull = (dfs.filter("profession <> r'\\N' and Titles <> r'\\N' ")
                 .dropDuplicates(["profession","Titles"]))
    return dffull


df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
tb = dp_etl(df)

tb.write.mode('overwrite').partitionBy("deathYear").parquet("/user/bigdata/repository/parquet/movie.parquet")

#spark.table("dbdev001.tb_movie").show()
#spark.sql("drop table if exists movie")
#tb.limit(1500).write.mode('overwrite').partitionBy("deathYear").saveAsTable("movie")
# tb.limit(1000).write.mode("overwrite").jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.movie",properties={"user": "root", "password": "mysql"})



In [ ]:
# spark.sql("show partitions movie").show()
# spark.sql("desc table extended movie").show()
# spark.table("movie").count()

df02 = spark.read.parquet("/user/bigdata/repository/parquet/movie.parquet")
df02.printSchema()




#3.449.518






#123.627.027
#11.845.112